In [5]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 32

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(os.getcwd(), 'CarneDataset/train'),seed =123,image_size=(IMG_SIZE,IMG_SIZE))
classes_train = train_ds.class_names
nClasses_train = len(classes_train)

Found 1633 files belonging to 8 classes.



Creamos el modelo de CNN con sus parametros

In [7]:
epochs = 10batch_size = 64
classifier_model = Sequential([
    tf.keras.layers.Rescaling(1./255,input_shape = (IMG_SIZE,IMG_SIZE,3)),
    Conv2D(16,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(32,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Conv2D(64,3,activation='relu',padding='same'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(128,activation='relu'),
    Dense(nClasses_train)
])
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
classifier_model.compile(optimizer=optimizer, 
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=["accuracy"])

Entrenamos el modelo CNN

In [8]:
classifier_train = classifier_model.fit(train_ds,batch_size=batch_size,epochs=epochs)

52/52 [==============================] - 1s 45ms/step - loss: 1.4075 - accuracy: 0.5713
Epoch 1/10
52/52 [==============================] - 1s 41ms/step - loss: 1.2174 - accuracy: 0.5903
Epoch 2/10
52/52 [==============================] - 1s 42ms/step - loss: 0.9015 - accuracy: 0.6816
Epoch 3/10
52/52 [==============================] - 1s 50ms/step - loss: 0.6664 - accuracy: 0.7446
Epoch 4/10
52/52 [==============================] - 1s 51ms/step - loss: 0.5447 - accuracy: 0.7924
Epoch 5/10
52/52 [==============================] - 1s 46ms/step - loss: 0.4778 - accuracy: 0.8026
Epoch 6/10
52/52 [==============================] - 1s 39ms/step - loss: 0.4268 - accuracy: 0.8530
Epoch 7/10
52/52 [==============================] - 1s 47ms/step - loss: 0.4766 - accuracy: 0.8700
Epoch 8/10
52/52 [==============================] - 1s 47ms/step - loss: 0.3934 - accuracy: 0.9030
Epoch 9/10
52/52 [==============================] - 1s 47ms/step - loss: 0.3648 - accuracy: 0.9230
Epoch 10/10
52/52 [==

Predicción para Test

In [10]:
datagen = ImageDataGenerator()
test_generator = datagen.flow_from_directory(os.path.join(os.getcwd(), 'CarneDataset/test'),
                                            target_size = (IMG_SIZE,IMG_SIZE),class_mode = 'categorical',shuffle=False)



predicted_test = classifier_model.predict(test_generator)

y_predict_test = np.argmax(predicted_test,axis=1)
y_real_test = test_generator.classes
correct_test = np.where(y_predict_test==y_real_test)[0]

print ('Correct test values %s from %s'%(len(correct_test),len(y_real_test)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_test)/len(y_real_test)*100))
confusion_matrix(y_real_test, y_predict_test)

Found 810 images belonging to 8 classes.


26/26 [==============================] - 2s 150ms/step
Correct test values 766 from 810
Porcentaje valores correctos: 90.12


array([[  0,   0,   0,   0,   0,   0,   1,   0],
       [  0,  40,   0,   9,   0,   0,   0,   0],
       [  0,   0,  73,   0,   2,   0,  20,   0],
       [  0,   2,   0,  43,   0,   1,   0,   0],
       [  0,   1,   5,   0, 441,   0,   4,   7],
       [  0,   0,   0,   0,   4,  15,   0,   0],
       [  0,   0,   8,   2,   4,   0, 100,   0],
       [  0,   0,   0,   0,   7,   0,   0,  20]])

In [12]:
train_generator = datagen.flow_from_directory(os.path.join(os.getcwd(), 'CarneDataset/train'),
                                            target_size = (IMG_SIZE,IMG_SIZE),class_mode = 'categorical',shuffle=False)

predicted_train = classifier_model.predict(train_generator)
y_predict_train = np.argmax(predicted_train,axis=1)
y_real_train = train_generator.classes
correct_train = np.where(y_predict_train==y_real_train)[0]

print ('Correct train values %s from %s'%(len(correct_train),len(y_real_train)))
print ('Porcentaje valores correctos: %.2f'%(len(correct_train)/len(y_real_train)*100))
confusion_matrix(y_real_train, y_predict_train)

52/52 [==============================] - 7s 127ms/step
Correct train values 1607 from 1633
Porcentaje valores correctos: 98.43


array([[ 61,   0,   2,   0,   0,   0,   0],
       [  0, 196,   0,   3,   0,  13,   0],
       [  0,   0, 104,   0,   0,   0,   0],
       [  0,   4,   0, 942,   0,   3,   1],
       [  0,   0,   0,   1,  35,   0,   0],
       [  1,   0,   1,   0,   0, 201,   0],
       [  0,   0,   0,   0,   0,   0,  62]])